In [68]:
import glob
import pandas as pd
import re

In [101]:
directorio_json = "/Users/rudikrompich/Git/analisis_tse_elecciones_2023/departamento/distrito_central/municipio/guatemala/corporacion_municipal/actas_procesadas"
directorio_json_documentos = glob.glob(directorio_json + "/*.json")
directorio_trep_documento = "/Users/rudikrompich/Git/analisis_tse_elecciones_2023/departamento/distrito_central/municipio/guatemala/corporacion_municipal/actas_csv_sistema_trep/4-CORPORACIÓN/gtm2023_e4d00m01.csv"


In [108]:
df_complete_ocr = pd.DataFrame()
for documento in directorio_json_documentos:
    parts = documento.split('/')
    desired_part = parts[-1]
    numbers = re.findall(r'\d+', desired_part)
    centro = int(numbers[0])
    mesa = int(numbers[1])
    df = pd.read_json(documento)
    df["CENTRO_DE_VOTACIÓN"] = centro
    df["MESA"] = mesa
    df_complete_ocr = pd.concat([df_complete_ocr, df])
df_complete_ocr.columns =  ['PARTIDO', 'CANTIDAD_OCR', 'CENTRO_DE_VOTACIÓN', 'MESA']
df_complete_ocr = df_complete_ocr[(df_complete_ocr["PARTIDO"]=="VALOR UNIONISTA") | (df_complete_ocr["PARTIDO"]=="COMPROMISO, RENOVACION Y ORDEN")].copy().reset_index(drop=True)
df_complete_ocr['PARTIDO'] = df_complete_ocr['PARTIDO'].replace('COMPROMISO, RENOVACION Y ORDEN', 'CREO')



In [112]:
df_trep_partial1 = pd.read_csv(directorio_trep_documento, sep=",",skiprows=4)

df_trep_partial1_1 = df_trep_partial1[["VALOR UNIONISTA", "MESA","CENTRO_DE_VOTACIÓN"]].copy()
df_trep_partial1_1.rename(columns={"VALOR UNIONISTA":"CANTIDAD"}, inplace=True)
df_trep_partial1_1["PARTIDO"] = "VALOR UNIONISTA"

df_trep_partial1_2 = df_trep_partial1[[ "CREO", "MESA","CENTRO_DE_VOTACIÓN"]].copy()
df_trep_partial1_2.rename(columns={"CREO":"CANTIDAD"}, inplace=True)
df_trep_partial1_2["PARTIDO"] = "CREO"

df_complete_trep = pd.concat([df_trep_partial1_1, df_trep_partial1_2]).reset_index(drop=True)
df_complete_trep.columns = ['CANTIDAD_TREP', 'MESA', 'CENTRO_DE_VOTACIÓN', 'PARTIDO']
df_complete_trep["CANTIDAD_TREP"] = df_complete_trep["CANTIDAD_TREP"].replace("--", 0)
df_complete_trep["CANTIDAD_TREP"] = df_complete_trep["CANTIDAD_TREP"].astype(int)

In [119]:
df_complete_ocr.groupby(["PARTIDO"]).sum()

,CANTIDAD_OCR,CENTRO_DE_VOTACIÓN,MESA
PARTIDO,,,
CREO,106323,152812,2047276
VALOR UNIONISTA,106570,152812,2047276


In [121]:
df_complete_trep.groupby(["PARTIDO"]).sum()


,CANTIDAD_TREP,MESA,CENTRO_DE_VOTACIÓN
PARTIDO,,,
CREO,105698,2047276,152812
VALOR UNIONISTA,106220,2047276,152812


In [126]:
df_completo_con_ocr_y_trep = pd.merge(df_complete_ocr, df_complete_trep, on=["CENTRO_DE_VOTACIÓN", "MESA", "PARTIDO"])
df_completo_con_ocr_y_trep

,PARTIDO,CANTIDAD_OCR,CENTRO_DE_VOTACIÓN,MESA,CANTIDAD_TREP
0,VALOR UNIONISTA,36,94,1276,36
1,CREO,67,94,1276,67
2,VALOR UNIONISTA,29,33,425,29
3,CREO,18,33,425,18
4,VALOR UNIONISTA,106,83,1093,106
...,...,...,...,...,...
4041,CREO,62,80,1022,62
4042,VALOR UNIONISTA,29,126,1782,29
4043,CREO,70,126,1782,70
4044,VALOR UNIONISTA,49,42,537,49


In [127]:
df_completo_con_ocr_y_trep["DIFERENCIA"] = df_completo_con_ocr_y_trep["CANTIDAD_OCR"] - df_completo_con_ocr_y_trep["CANTIDAD_TREP"]

In [130]:
df_final_diferencias = df_completo_con_ocr_y_trep[df_completo_con_ocr_y_trep["DIFERENCIA"] != 0].copy()


In [132]:
df_final_diferencias.to_csv("/Users/rudikrompich/Git/analisis_tse_elecciones_2023/departamento/distrito_central/municipio/guatemala/corporacion_municipal/diferencias_ocr_trep/diferencias.csv", index=False)